In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy

import import_ipynb
import ResNetCaps

verbose = False
USE_CUDA = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


CIFAR10_USE = True
verbose = False

In [ ]:
dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


batch_size = 100


if CIFAR10_USE: 
    NUM_CLASSES = 10
    print("CIFAR10")
    image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

In [ ]:
model = ResNetCaps.ResNetCaps(10)
model = model.to(device)

optimizer = optim.Adam(model.model.layer3.parameters(),lr = 0.001)
start = time.time()

batch_id = 100
inputs, labels = next(iter(dataloaders['train']))

train_loss = 0
train_accuracy = 0

    
labels =torch.eye(10).index_select(dim=0, index=labels)
inputs, labels = Variable(inputs), Variable(labels)
inputs = inputs.to(device)
labels = labels.to(device)

optimizer.zero_grad()

outputs, masked = model.forward(inputs)
loss = model.model_loss(outputs, labels)


loss.backward()
optimizer.step()

train_loss += loss.data[0]
train_accuracy += (sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

if batch_id % 100 == 0:
    print("train accuracy:", sum(np.argmax(masked.data.cpu().numpy(), 1) == 
                           np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
    if verbose: print("masked {}".format(np.argmax(masked.data.cpu().numpy(), 1)))
    if verbose: print("labels {}".format(np.argmax(labels.data.cpu().numpy(), 1)))
#                batch_accuracy.append(sum(np.argmax(preds.data.cpu().numpy(), 1) == 
#                                       np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy / len(dataloaders['train'])))